#### 残差块定义

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

class Residual(nn.Module):
    def __init__(self,input_channels,num_channels,use_1x1conv=False,strides=1):
        super().__init__()
        self.conv1=nn.Conv2d(input_channels,num_channels,kernel_size=3,padding=1,stride=strides)
        self.conv2=nn.Conv2d(num_channels,num_channels,kernel_size=3,padding=1)
        if use_1x1conv:
            self.conv3=nn.Conv2d(input_channels,num_channels,kernel_size=1,stride=strides)
        else:
            self.conv3=None
        self.bn1=nn.BatchNorm2d(num_channels)
        self.bn2=nn.BatchNorm2d(num_channels)

    def forward(self,X):
        Y=F.relu(self.bn1(self.conv1(X)))
        Y=self.bn2(self.conv2(Y))
        if self.conv3:
            X=self.conv3(X)
        Y+=X
        return F.relu(Y)

### ResNet模型
#### 可以理解为3个部分：残差块定义 + 残差模块生成函数 + ResNet完整架构

#### 残差模块生成函数

In [ ]:
def resnet_block(input_channels,num_channels,num_residuals.first_block=False):
    blk=[]
    # 情况1：不是第一个残差模块的第一个残差块。通道翻倍+尺寸减半
    for i in range(num_channels):
        if i==0 and not first_block:
            blk.append(Residual(input_channels,num_channels,use_1x1conv=True,strides=2))
    # 情况2：第一个残差模块的第一个残差块，保持形状不变
        else:
            blk.append(Residual(num_channels,num_channels))
    return blk

#### ResNet完整架构

In [ ]:
# 初始卷积 + 池化
b1=nn.Sequential(
    nn.Conv2d(1,64,kernel_suize=7,stride=2,padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2,padding=1)#尺寸减半
)
# 4个残差模块（b2-b5）：核心特征提取
b2=nn.Sequential(*resnet_block(64, 64, 2, first_block=True))# 第一个模块：通道和尺寸不变
b3 = nn.Sequential(*resnet_block(64, 128, 2))  # 通道64→128，尺寸减半
b4 = nn.Sequential(*resnet_block(128, 256, 2)) # 通道128→256，尺寸减半
b5 = nn.Sequential(*resnet_block(256, 512, 2)) # 通道256→512，尺寸减半
# 全局池化+全连接层
b6=nn.Sequential(
    nn.AdaptiveAvgPool2d((1,1)), # 全局平均池化：[B,512,H,W] → [B,512,1,1]
    nn.Flatten(), # 展平：[B,512,1,1] → [B,512]
    nn.Linear(512,10)# 全连接：512维特征 → 10类输出
)
# 组装完整模型
net = nn.Sequential(b1, b2, b3, b4, b5, b6)